In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
use schema PYTHON_WORKLOADS.BRIANS;

In [ ]:
CREATE OR REPLACE function get_qualtrics_payroll_api_key()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.8
HANDLER = 'get_secret'
EXTERNAL_ACCESS_INTEGRATIONS = (qualtrics_apis_access_integration)
SECRETS = ('cred' = LHPY_QUALTRICS_APITOKEN )
AS
$$
import _snowflake
def get_secret():
  secret_type = _snowflake.get_generic_secret_string('cred')
  return secret_type
$$;



In [ ]:
select PYTHON_WORKLOADS.BRIANS.get_qualtrics_payroll_api_key() as APIKEY

In [ ]:
api_key_from_secret=token_from_secret.to_pandas()['APIKEY'].iloc[0]

In [ ]:
# import requests
# dataCenter = 'az1'
# surveyId = 'SV_6hFfHRKWEqv17Ya'
# apiToken = 'yXgRlDN2T6mgFZnOZ0sOgZaXCfCvX94vsps0hxiW'

# #step 1 - Pull distribution ids associated with the chosen
# #adding test comment for SDLC demo 14dec22_1

# nextPage = "https://{0}.qualtrics.com/API/v3/distributions?surveyId={1}".format(dataCenter, surveyId)
# headers = {
#         "content-type": "application/json",
#         "x-api-token": apiToken,
#         }
# #create a list to load initial API results into. Each API pull brings in JSON code that python reads as a dictionary.
# distribution_list=[]
# distribution_id_list=[]
# distribution_dic={}

# r=requests.get(nextPage, headers=headers, verify=False)
# print(r)
# results=r.text
# print(results)

In [ ]:
select *, 1 as is_implementation from bi.ci.LHPY_PAYROLL_IMPLEMENTATION_SURVEY limit 500


In [ ]:
select *, 1 as is_annual from bi.ci.LHPY_PAYROLL_ANNUAL_SURVEY limit 500

In [ ]:
select *, 1 as is_exit from bi.ci.LHPY_PAYROLL_EXIT_SURVEY limit 500

In [ ]:
select *, 1 as is_client_services from bi.ci.LHPY_PAYROLL_CLIENT_SERVICES_SURVEY limit 10000

In [ ]:
select *, 1 as is_retention from bi.ci.LHPY_PAYROLL_PAYROLL_RETENTION_SURVEY limit 500

In [ ]:
select *, 1 as is_case from bi.ci.LHPY_PAYROLL_CASE_TAA_SURVEY limit 500

In [ ]:
select *, 1 as is_no_start from bi.ci.LHPY_PAYROLL_NO_START_SURVEY limit 100

In [ ]:
import os
#os.chdir('/content/drive/MyDrive/Qualtrics/sendsurveys')

#!pip install snowflake-connector-python

#libraries
import requests
import snowflake.connector
import csv
import sys
import os
import datetime
import tempfile






apiToken=api_key_from_secret
dataCenter = "pdx1"

print(apiToken)

headers = { "x-api-token": apiToken }

def left(s, amount):
    return s[:amount]

qualtrics_files = [

{        #qualitrics ID
        'automationId': 'AU_2UepKn9ihgAljZj',
        'file_name': 'lhpy_payroll_post_implementation_survey',
        'table': 'bi.ci.LHPY_PAYROLL_IMPLEMENTATION_SURVEY'
}
 ,
   {
       'automationId': 'AU_rJaYRkeRasRyW9i',
       'file_name': 'lhpy_payroll_annual_relationship_survey',
       'table': 'bi.ci.LHPY_PAYROLL_ANNUAL_SURVEY'
   },

   {
       'automationId': 'AU_YlDZ9r7QnzRfEsj',
       'file_name': 'lhpy_payroll_exit_survey',
       'table': 'bi.ci.LHPY_PAYROLL_EXIT_SURVEY'
   }
  ,

   {
       'automationId': 'AU_LqjnR0wLhWeTtUt',
       'file_name': 'LHPY_PAYROLL_CLIENT_SERVICES_SURVEY',
       'table': 'bi.ci.LHPY_PAYROLL_CLIENT_SERVICES_SURVEY'
   }
 ,

   {
       'automationId': 'AU_QsaZV7pLr9Hp3gc',
       'file_name': 'LHPY_PAYROLL_PAYROLL_RETENTION_SURVEY',
       'table': 'bi.ci.LHPY_PAYROLL_PAYROLL_RETENTION_SURVEY'
   }
  ,

   {
       'automationId': 'AU_8k0HVw48XzMZK1l',
       'file_name': 'LHPY_PAYROLL_CASE_TAA_SURVEY',
       'table': 'bi.ci.LHPY_PAYROLL_CASE_TAA_SURVEY'
   }
    ,

   {
       'automationId': 'AU_gSOCP6EoRSxWNEq',
       'file_name': 'LHPY_PAYROLL_NO_START_SURVEY',
       'table': 'bi.ci.LHPY_PAYROLL_NO_START_SURVEY'
   }

]

def left(s, amount):
    return s[:amount]
#left(str(datetime.datetime.now()), 10).replace('-','')




for file in qualtrics_files:
    automationId = file['automationId']
    file_name = file['file_name']
    print(file_name)
    
    sql = "SELECT * FROM {0}".format(file['table'])
    
    current_date = left(str(datetime.datetime.now()), 10).replace('-','')

    #baseUrl = "https://heartlandcx.{0}.qualtrics.com/automations-file-service/automations/{1}/files".format(dataCenter, automationId)
    
    # baseUrl = "https://sjc1.qualtrics.com/automations-file-service/automations/{0}/files".format(automationId)
    baseUrl = "https://{0}.qualtrics.com/automations-file-service/automations/{1}/files".format(dataCenter,automationId)
    f = tempfile.NamedTemporaryFile(delete=False)
    temp_file_path= f.name

    #csv_file_path = '/content/drive/MyDrive/Qualtrics/sendsurveys/{0}.csv'.format(file_name)
    #csv_archive_file_path = '/content/drive/MyDrive/Qualtrics/sendsurveys/Archive/{0}.csv'.format(file_name + '_' + current_date)

    #csv_file_path = '/Users/Meghana.Nagendra/Desktop/{0}'.format(file_name)
    #csv_archive_file_path = '/Users/Meghana.Nagendra/Desktop/qualtrics_archive/archive/{0}.csv'.format(file_name + '_' + current_date)
    
    print ('Sending', file_name, temp_file_path)

    # cursor.execute(sql)
    # rows = cursor.fetchall()
    if file_name=='lhpy_payroll_post_implementation_survey':
        rows = LYPY_PAYROLL_IMPLEMENTATION_SURVEY.to_pandas()
    elif file_name=='lhpy_payroll_annual_relationship_survey':
        rows = LHPY_PAYROLL_ANNUAL_SURVEY.to_pandas()
    elif file_name=='lhpy_payroll_exit_survey':
        rows = LHPY_PAYROLL_EXIT_SURVEY.to_pandas()
    elif file_name=='LHPY_PAYROLL_CLIENT_SERVICES_SURVEY':
        rows = LHPY_PAYROLL_CLIENT_SERVICES_SURVEY.to_pandas()
    elif file_name=='LHPY_PAYROLL_PAYROLL_RETENTION_SURVEY':
        rows = LHPY_PAYROLL_PAYROLL_RETENTION_SURVEY.to_pandas()
    elif file_name=='LHPY_PAYROLL_CASE_TAA_SURVEY':
        rows = LHPY_PAYROLL_CASE_TAA_SURVEY.to_pandas()
    elif file_name=='LHPY_PAYROLL_NO_START_SURVEY':
        rows = LHPY_PAYROLL_NO_START_SURVEY.to_pandas()
        
    # if len(rows)>0:
    #     result = list()
    #     column_names = list()
    #     print(rows)
    #     for row in rows:
    #         column_names.append(row)
    #     print(column_names)

    #     for index, record in rows.iterrows():
    #         result.append(record)
    #     print(result)
    if not rows.empty:
        rows.to_csv(temp_file_path, sep='|',quotechar='"',quoting=csv.QUOTE_MINIMAL, index=False, encoding='UTF-8')
    #     for row in rows:
    #         result.append(row)

    #     with open(temp_file_path, 'w', newline='') as csvfile:
    #         csvwriter = csv.writer(csvfile, delimiter='|', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #         for row in result:
    #             csvwriter.writerow(row)

        with open(temp_file_path, 'r',encoding='UTF-8') as uploadfile:
            response = requests.post(baseUrl, files={ 'file': uploadfile }, headers=headers, verify=False)
            print(response.text)
            print(uploadfile)
            print('would of sent :', uploadfile)

    #   #  os.rename(temp_file_path, csv_archive_file_path)

    # else:
    #     print("No rows found for query: {0}".format(sql))